In [18]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors

Використовуємо дані з файлів bda_dataset5_2_rating.csv і bda_dataset5_2_book.csv

In [3]:
rating_df = pd.read_csv("bda_dataset5_2_rating.csv")
book_df = pd.read_csv("bda_dataset5_2_book.csv")
rating_df.head()

C:\Users\HP\AppData\Local\Temp\ipykernel_12680\2300945186.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  book_df = pd.read_csv("bda_dataset5_2_book.csv")


,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [4]:
book_df.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


Створюємо матрицю користувач-книга

рядки = користувачі

стовпці = книги

значення = рейтинг 

In [ ]:
def create_matrix(df):
    N = len(df['User-ID'].unique())
    M = len(df['ISBN'].unique())

    # ІD → індекси
    user_mapper = dict(zip(np.unique(df["User-ID"]), list(range(N))))
    book_mapper = dict(zip(np.unique(df["ISBN"]), list(range(M))))

    # Індекси → ІD (навпаки)
    user_inv_mapper = dict(zip(list(range(N)), np.unique(df["User-ID"])))
    book_inv_mapper = dict(zip(list(range(M)), np.unique(df["ISBN"])))

    # Перетворюємо user/book ID в індекси для побудови матриці
    user_index = [user_mapper[i] for i in df['User-ID']]
    book_index = [book_mapper[i] for i in df['ISBN']]

    # Створюємо розріджену матрицю: книги (рядки) × користувачі (стовпці)
    X = csr_matrix((df["Book-Rating"], (book_index, user_index)), shape=(M, N))

    return X, user_mapper, book_mapper, user_inv_mapper, book_inv_mapper

In [24]:
def get_similar_books(target_isbn, X, book_mapper, book_inv_mapper, book_df):
    # Індекс цільової книги у матриці
    target_idx = book_mapper[target_isbn]

    # Обчислюємо косинусну схожість
    similarities = cosine_similarity(X[target_idx], X).flatten()

    # Отримуємо топ-10 подібних книг (крім самої себе)
    similar_indices = similarities.argsort()[::-1][1:10+1]

    # Збираємо рекомендації
    recommendations = []
    for idx in similar_indices:
        isbn = book_inv_mapper[idx]
        title = book_df.loc[book_df['ISBN'] == isbn, 'Book-Title'].values
        author = book_df.loc[book_df['ISBN'] == isbn, 'Book-Author'].values
        sim_score = similarities[idx]

        recommendations.append({
            'ISBN': isbn,
            'Title': title[0] if len(title) > 0 else 'Unknown',
            'Similarity': round(sim_score, 3)
        })

    return pd.DataFrame(recommendations)


In [25]:
target_isbn = "0060973129"  
X, user_mapper, book_mapper, user_inv_mapper, book_inv_mapper = create_matrix(rating_df)
similar_books_df = get_similar_books(target_isbn, X, book_mapper, book_inv_mapper, book_df)

similar_books_df

,ISBN,Title,Similarity
0,1853673110,War on the Eastern Front: The German Soldier i...,0.753
1,0804100039,If You Survive: From Normandy to the Battle of...,0.753
2,0891415947,August 1944: The Campaign for France,0.753
3,0743201981,Always Faithful : A Memoir of the Marine Dogs ...,0.753
4,0440236304,Currahee!: A Screaming Eagle at Normandy,0.753
5,0380973499,War's End: An Eyewitness Account of America's ...,0.753
6,0836249348,Cc Nutcracker,0.659
7,0771029772,Ignorant Armies : Sliding into War in Iraq,0.659
8,0764108875,101 Questions Your Cat Would Ask: What's Bothe...,0.659
9,0385319622,"American Chica: Two Worlds, One Childhood",0.626


In [21]:
def get_knn_recommendations(isbn, X, book_mapper, book_inv_mapper, book_df, k=10):
    if isbn not in book_mapper:
        return pd.DataFrame(columns=["ISBN", "Title", "Author", "Similarity"])

    book_idx = book_mapper[isbn]

    model = NearestNeighbors(metric='cosine', algorithm='brute')
    model.fit(X)

    # Обмеження для k, щоб не перевищувати кількість книг
    n_books = X.shape[0]
    k = min(k + 1, n_books)

    distances, indices = model.kneighbors(X[book_idx], n_neighbors=k)

    similar_books = []
    for i in range(1, len(distances[0])):  # пропускаємо саму книгу
        idx = indices[0][i]
        dist = distances[0][i]
        similar_isbn = book_inv_mapper[idx]
        row = book_df[book_df['ISBN'] == similar_isbn]
        title = row['Book-Title'].values[0] if not row.empty else "Unknown"
        author = row['Book-Author'].values[0] if not row.empty else "Unknown"
        similar_books.append({
            "ISBN": similar_isbn,
            "Title": title,
            "Similarity": 1 - dist
        })

    return pd.DataFrame(similar_books)

In [23]:
target_isbn = "0060973129"  
X, user_mapper, book_mapper, user_inv_mapper, book_inv_mapper = create_matrix(rating_df)
similar_books_df = get_knn_recommendations(target_isbn, X, book_mapper, book_inv_mapper, book_df)

similar_books_df

,ISBN,Title,Similarity
0,1853673110,War on the Eastern Front: The German Soldier i...,0.752577
1,0804100039,If You Survive: From Normandy to the Battle of...,0.752577
2,0891415947,August 1944: The Campaign for France,0.752577
3,0380973499,War's End: An Eyewitness Account of America's ...,0.752577
4,0440236304,Currahee!: A Screaming Eagle at Normandy,0.752577
5,0743201981,Always Faithful : A Memoir of the Marine Dogs ...,0.752577
6,0836249348,Cc Nutcracker,0.658505
7,0764108875,101 Questions Your Cat Would Ask: What's Bothe...,0.658505
8,0771029772,Ignorant Armies : Sliding into War in Iraq,0.658505
9,0385319622,"American Chica: Two Worlds, One Childhood",0.626182


Додатково виводимо рекомендацію 10 книг для конкретного користувача на основі книги, якій він (чи вона) присвоїв (-ла) максимальний рейтинг

In [28]:
def get_recs_for_user(user_id, X, book_mapper, book_inv_mapper, book_df, rating_df, k=10):
    user_ratings = rating_df[rating_df["User-ID"] == user_id]
    if user_ratings.empty:
        return None, None, pd.DataFrame()

    max_rating = user_ratings["Book-Rating"].max()
    top_books = user_ratings[user_ratings["Book-Rating"] == max_rating]

    target_isbn = top_books.iloc[0]["ISBN"]
    book_title = book_df[book_df["ISBN"] == target_isbn]["Book-Title"].values[0] if not book_df[book_df["ISBN"] == target_isbn].empty else "Unknown"

    similar_books_df = get_knn_recommendations(target_isbn, X, book_mapper, book_inv_mapper, book_df, k)

    return target_isbn, book_title, similar_books_df


In [29]:
user_id = 276727 
X, user_mapper, book_mapper, user_inv_mapper, book_inv_mapper = create_matrix(rating_df)
target_isbn, book_title, recs = get_recs_for_user(user_id, X, book_mapper, book_inv_mapper, book_df, rating_df)

print(f"На основі книги: {book_title} (ISBN: {target_isbn})")
recs

На основі книги: The Notebook (ISBN: 0446520802)


,ISBN,Title,Similarity
0,0142300373,A Midnight Clear: Family Christmas Stories,0.208665
1,0671750127,Until Proven Guilty,0.198941
2,0451128559,The Shoemaker : The Anatomy of a Psychotic,0.197743
3,0679894608,The Lucky Lottery (A to Z Mysteries (Paper)),0.197167
4,0307988414,Disney's Pooh's Grand Adventure the Search for...,0.194543
5,0316710571,Sam's Letters to Jennifer,0.193193
6,0316120049,Handbook for the Heart : Original Writings on ...,0.189402
7,0684801884,Women Make the Best Friends: A Celebration,0.185203
8,0552138959,Unknown,0.177500
9,0752814362,Unknown,0.175771
